### ⭕ **Ejercicio importante por tu cuenta**

Prueba el modelo del ejercicio dos con varias modificaciones y revisamos en cada caso:
1. El rendimiento (R2/MAE/MSE), ¿disminuye o aumenta?
2. ¿Qué variables son las que son más importantes?

* Cargamos la info:

In [29]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


url = 'https://github.com/DCDPUAEM/DCDP/raw/main/03%20Machine%20Learning/data/cars-prices.csv'

* Procesamos el dataset:

In [57]:
##PROCESAMIENTO
#quitamos algunas variables no relevantes
df = pd.read_csv(url)
df.drop(columns=['car_ID','CarName'],inplace=True)

# Reemplazar los datos de las columnas `cylindernumber` y `doornumber` con número. Las convertimos a variables numéricas
cols = ['cylindernumber','doornumber']
replacing_dict = {'four':4, 'six':6, 'five':5, 'three':3, 'twelve':12, 'two':2, 'eight':8}
for col in cols:
    df[col].replace(replacing_dict,inplace=True)

#SEPARAMOS VARIABLES DE SALIDA
y = df['price'].values
df.drop(columns='price',inplace=True)

#CODIGICACION ONE HOT PARA VARIABLES CATEGÓRICAS
X_df = pd.get_dummies(df)

#DEFINIMOS MATRIZ DE FEATURES
X = X_df.values

#Dividimos los datos en entrenamiento (train) y prueba (test) con una proporción 80%/20%.
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=4595)
print(f"Entrenamiento: {X_train.shape,y_train.shape}")
print(f"Prueba: {X_test.shape,y_test.shape}")

Entrenamiento: ((164, 45), (164,))
Prueba: ((41, 45), (41,))


#### 1. *¿Qué pasa si no usas normalización?*

* Probamos el modelo sin normalización:

In [73]:
selector = VarianceThreshold()
X_train = selector.fit_transform(X_train)   # Entrenamos y transformamos el de entrenamiento
X_test = selector.transform(X_test)        # Sólo transformamos el de prueba

### NO REALIZAMOS LA NORMALIZACION PARA ESTOS DATOS
lr_1 = LinearRegression()
lr_1.fit(X_train, y_train)
y_pred_test_1 = lr_1.predict(X_test)
y_pred_train_1 = lr_1.predict(X_train)

#CREAMOS COPIA PARA HACERLO CON DATOS NORMALIZADOS
X_train_2=X_train.copy()
X_test_2=X_test.copy()

scaler = StandardScaler()
scaler = MinMaxScaler()
X_train_2 = scaler.fit_transform(X_train_2)
X_test_2 = scaler.transform(X_test_2)
lr_2 = LinearRegression()
lr_2.fit(X_train_2, y_train)

y_pred_test_2 = lr_2.predict(X_test_2)
y_pred_train_2 = lr_2.predict(X_train_2)

#COMPARAMOS
print(f'R2 entrenamiento [NO NORMALIZADO: {lr_1.score(X_train,y_train).round(2)}, NORMALIZADOS: {lr_2.score(X_train_2,y_train).round(2)}]')
print(f'R2 prueba [NO NORMALIZADO: {lr_1.score(X_test,y_test).round(2)}, NORMALIZADOS: {lr_2.score(X_test_2,y_test).round(2)}]\n')

print(f"MAE entrenamiento [NO NORMALIZADO: {mean_absolute_error(y_train,y_pred_train_1).round(2)}, NORMALIZADOS:  {mean_absolute_error(y_train,y_pred_train_2).round(2)}]")  # Esta es muy interpretativa
print(f"MAE prueba [NO NORMALIZADO: {mean_absolute_error(y_test,y_pred_test_1).round(2)}, NORMALIZADOS: {mean_absolute_error(y_test,y_pred_test_2).round(2)}]\n")  # Esta es muy interpretativa

print(f"MSE entrenamiento [NO NORMALIZADO: {mean_squared_error(y_train,y_pred_train_1).round(2)}, NORMALIZADOS: {mean_squared_error(y_train,y_pred_train_2).round(2)}]")
print(f"MSE prueba [NO NORMALIZADO: {mean_squared_error(y_test,y_pred_test_1).round(2)}, NORMALIZADOS: {mean_squared_error(y_test,y_pred_test_2).round(2)}]\n")


columns = X_df.columns.to_list()
coefs_dict = dict(zip(columns,lr_1.coef_))
print(f'Principales variables para determinar el precio con valores no normalizados: \n{dict(sorted(coefs_dict.items(),key=lambda x:x[1],reverse=True)[0:5])}\n')

columns = X_df.columns.to_list()
coefs_dict = dict(zip(columns,lr_2.coef_))
print(f'Principales variables para determinar el precio con valores normalizados: \n{dict(sorted(coefs_dict.items(),key=lambda x:x[1],reverse=True)[0:5])}')

R2 entrenamiento [NO NORMALIZADO: 0.93, NORMALIZADOS: 0.93]
R2 prueba [NO NORMALIZADO: 0.89, NORMALIZADOS: 0.89]

MAE entrenamiento [NO NORMALIZADO: 1478.95, NORMALIZADOS:  1478.95]
MAE prueba [NO NORMALIZADO: 2234.67, NORMALIZADOS: 2234.67]

MSE entrenamiento [NO NORMALIZADO: 3692070.72, NORMALIZADOS: 3692070.72]
MSE prueba [NO NORMALIZADO: 11944281.56, NORMALIZADOS: 11944281.56]

Principales variables para determinar el precio con valores no normalizados: 
{'enginetype_rotor': 7769.116621714023, 'enginelocation_rear': 5703.189008174585, 'fuelsystem_idi': 5295.5601553045335, 'fueltype_diesel': 5295.560155304531, 'carbody_convertible': 2901.1564386774576}

Principales variables para determinar el precio con valores normalizados: 
{'enginesize': 32482.93170169278, 'curbweight': 11712.045795038339, 'enginetype_rotor': 7769.116621714724, 'carwidth': 6676.770950541305, 'enginelocation_rear': 5703.189008173684}


**Comentarios:**  
Con estos resultados podemos observar que entre los valores normalizados y los no normalizados no hay una diferencia en los índices de rendimiento, pero si se ven afectadas las principales variables para determinar el valor del precio.  
Con los datos no normalizados tenemos que **enginetype_rotor** y **enginelocation_rear** son las variables con mayor peso. Mientras que con los datos normalizados tenemos que son **enginesize** y **curbweight**

####  2. *¿Qué pasa si no haces selección de features? ¿qué pasa si sólo te quedas con algunas features seleccionadas *intuitivamente*?*


#### 3. *¿Qué pasa si no cuidas el data leakage? Es decir, haz primero todo el preprocesamiento y después divides en train/test.*


Una pregunta que surge con frecuencia se refiere a la incertidumbre acerca de los parámetros del modelo, ya que estos también son variables aleatorias.

En general, Scikit-Learn no proporciona herramientas para obtener conclusiones acerca de los parámetros internos del modelo: la interpretación de los parámetros del modelo es mucho más una *pregunta de modelaje estadístico* que una pregunta de *aprendizaje automático*. El aprendizaje automático se centra más la *predicción*.

No obstante, si se desea profundizar en el significado de los parámetros del modelo, hay herramientas como las incluidas en el [paquete de Statsmodels de Python](http://statsmodels.sourceforge.net/).

In [ ]:
import statsmodels.api as sm

X_train_1 = sm.add_constant(X_train)
X_train_1_df = pd.DataFrame(X_train_1, columns=['const']+X_df.columns.to_list())
display(X_train_1_df.head())
results = sm.OLS(y_train, X_train_1_df).fit()
print(results.summary())

**Algunas observaciones**

* `R-squared` nos dice que nuestro modelo explica 93% del cambio en la variable dependiente (el precio del automovil).
* `Adj. R-squared` ajusta el coeficiente `R-squared` tomando en cuenta el número de variables (features).
* Los p-values `P>|t|` nos dice qué tan probable es que el coeficiente de la población haya sido medido por azar. Es decir, pone a prueba la hipótesis nula de que la variable independiente no tiene correlación con la variable dependiente. Hay dos escenarios:
    * El valor es superior al nivel de significancia, no hay asociación entre los cambios en la variable independiente y los cambios en la variable dependiente. En otras palabras, no hay pruebas suficientes para concluir que existe un efecto a nivel de población.
    * Si el valor p de una variable es inferior al nivel de significancia, los datos de la muestra proporcionan pruebas suficientes para rechazar la hipótesis nula de toda la población. Sus datos favorecen la hipótesis de que existe una correlación distinta de cero. Los cambios en la variable independiente están asociados a cambios en la variable dependiente a nivel poblacional. Esta variable es estadísticamente significativa y probablemente merezca la pena añadirla a su modelo de regresión.

## Ejemplo 3: Regularización

La regularización es una técnica de regresión lineal que penaliza la magnitud de los coeficientes de la regresión. Los coeficientes minimizan una suma de cuadrados residual penalizada:

$$\text{min}_w \|Xw - y \|^2+α\|w\|^2$$

El parámetro de complejidad $\alpha>0$ controla la cantidad de contracción: cuanto mayor sea el valor de $\alpha$, mayor será la cantidad de contracción.

Hay tres tipos de regularización en la regresión:

* [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html): Es la descrita anteriormente.
* [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html): Es parecida a la anterior, pero usando la norma L1:
    $$\| w \| _1 = \sum |w_i|$$
* [ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet): Combina ambas anteriores.

En esto caso, sólo experimentaremos con la primera y veremos:

1. El efecto en la norma de los coeficientes.
2. El efecto en el rendimiento de la tarea de regresión.

En notebooks posteriores analizaremos un poco más a fondo la regularización.

Repetimos el procedimiento, ya de forma más compacta:

In [ ]:
import pandas as pd

url = 'https://github.com/DCDPUAEM/DCDP/raw/main/03%20Machine%20Learning/data/cars-prices.csv'
df = pd.read_csv(url)
df.drop(columns=['car_ID','CarName'],inplace=True)
cols = ['cylindernumber','doornumber']
replacing_dict = {'four':4, 'six':6, 'five':5, 'three':3, 'twelve':12, 'two':2, 'eight':8}

for col in cols:
    df[col].replace(replacing_dict,inplace=True)
df

In [ ]:
y = df['price'].values

df.drop(columns='price',inplace=True)
X_df = pd.get_dummies(df)
X_df.head()

In [ ]:
X = X_df.values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=4595)

print(f"Entrenamiento: {X_train.shape,y_train.shape}")
print(f"Prueba: {X_test.shape,y_test.shape}")

In [ ]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold()
X_train = selector.fit_transform(X_train)   # Entrenamos y transformamos el de entrenamiento
X_test = selector.transform(X_test)        # Sólo transformamos el de prueba

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StandardScaler()
# scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import Ridge

lr = Ridge(alpha=2)
lr.fit(X_train,y_train)

print(f"Entrenamiento: {lr.score(X_train,y_train)}")
print(f"Prueba: {lr.score(X_test,y_test)}")

In [ ]:
columns = X_df.columns.to_list()
coefs_dict = dict(zip(columns,lr.coef_))
dict(sorted(coefs_dict.items(),key=lambda x:x[1],reverse=True))

¿Qué efecto tiene el parámetro $\alpha$ sobre la magnitud de los coeficientes?

In [ ]:
import matplotlib.pyplot as plt

alphas = np.logspace(-12,0,num=10)
normas = []

for alpha in alphas:
    lr = Ridge(alpha=alpha)
    lr.fit(X_train,y_train)
    normas.append(np.mean(np.abs(lr.coef_)))

plt.figure()
plt.plot(alphas,normas)
plt.xlabel("alpha")
plt.ylabel("Norma promedio")
plt.show()

¿Qué efecto tiene el parámetro $\alpha$ en el score?

In [ ]:
import matplotlib.pyplot as plt

alphas = np.logspace(-10,0,num=10)
train_scores = []
test_scores = []

for alpha in alphas:
    lr = Ridge(alpha=alpha)
    lr.fit(X_train,y_train)
    train_scores.append(lr.score(X_train,y_train))
    test_scores.append(lr.score(X_test,y_test))

plt.figure()
plt.plot(alphas,train_scores,label="Train")
plt.plot(alphas,test_scores,label="Test")
plt.xlabel("alpha")
plt.ylabel("score")
plt.legend()
plt.show()

Como podemos ver, en este ejemplo concreto, la regularización no beneficia al problema en cuestión de la métrica de rendimiento.

#⭕ Ejercicio

Usaremos un dataset sobre publicidad. Este dataset consta de 200 registros, cada registro consta de las variables.

* TV: dólares de publicidad gastados en TV para un solo producto en un mercado determinado (en miles de dólares)
* Radio: inversión publicitaria en radio
* Newspaper: inversión publicitaria en periódicos
* Sales: ventas de un solo producto en un mercado determinado (en miles de unidades).


In [ ]:
import pandas as pd

url = 'https://github.com/DCDPUAEM/DCDP/raw/main/03%20Machine%20Learning/data/advertising.csv'
df = pd.read_csv(url,index_col=0)
df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.pairplot(df, x_vars=['TV','Radio','Newspaper'], y_vars='Sales', height=5, aspect=1, kind='reg')
plt.show()

Objetivos:

1. Entrenar un modelo de regresión lineal usando el 85% de las instancias, separar el resto para prueba. **No olvides el preprocesamiento**, **cuidado con el data leakage**.
2. Reportar las métricas de rendimiento MAE, MSE en las predicciones con el conjunto de prueba solamente.
3. Con base en la métrica de rendimiento MAE, escoge el mejor modelo de regresión lineal. Es decir, ¿cuál es el menor MAE que puedes obtener en el conjunto de prueba? Junto con este número, reporta los parámetros y la combinación de técnicas que usaste.

Considera las siguientes situaciones en el preprocesamiento:

* ¿Hay valores faltantes? En caso de que sí, recuerda que tienes dos opciones: remover estas instancias o hacer imputación.
* ¿Cuál es el rango de las 3 variables? ¿tienen magnitudes muy diferentes?
* ¿Hay alguna variable que consideres que no es muy relevante?
* Realiza la(s) técnica(s) de normalización que consideres necesario: selección de features, normalización.


